In [1]:
# This should be set by Pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.ui.enabled", False).getOrCreate()

In [2]:
# Initialise test environment
from dapla.magics import DaplaLineageMagics
from IPython import get_ipython
ipython = get_ipython()
from dapla.jupyterextensions.authextension import AuthClient
import requests
import json


# Lineage template provider can either call a local endpoint or read from attached json file
use_mock_response = True

def lineage_template_provider(output_schema, input_schema_map):
    def mapper(x):
        return (x[0], {
            "schema": x[1]['schema'],
            "schemaType": "SPARK",
            "timestamp": x[1]['timestamp'],
        })
    request = {
        "schema": output_schema['schema'],
        "timestamp": output_schema['timestamp'],
        "schemaType": "SPARK",
        "simpleLineage": False,
        "dependencies": [dict(map(mapper, input_schema_map.items()))],
    }
    if use_mock_response:
        with open('lineage-template.json', 'r') as f:
            return json.load(f)
    else:
        response = requests.post('http://localhost:10190/lineage/template', json=request,
                                 headers={
                                 }, allow_redirects=False)
        return response.json()
    
# Register dapla magics manually
magics = DaplaLineageMagics(ipython, lineage_template_provider)
ipython.register_magics(magics)

In [3]:
from pyspark.sql.types import *

# Create 3 test dataframe
person_type = StructType([
    StructField('personidentifikator', StringType()),
    StructField('kontonummer', StringType())])
person_data = [
    ('1234', '12345555'),
    ('1235', '12347777'),
]
person = spark.createDataFrame(person_data, person_type)

unrelated_type = StructType([
    StructField('weird', StringType()),
    StructField('stuff', StringType())])
unrelated = spark.createDataFrame([], unrelated_type)


konto_type = StructType([
    StructField('kontonummer', StringType()),
    StructField('innskudd', IntegerType())])
konto_data = [
    ('12345555', 25000),
    ('12347777', 120000),
]
konto = spark.createDataFrame(konto_data, konto_type)

# Create a 3rd dataframe based on the other two
innskudd = person.join(konto, 'kontonummer', how='inner')

In [4]:
# This will be done automatically by spark.read.path

%lineage_input /skatt/person {person.schema.json()}
%lineage_input /skatt/konto {konto.schema.json()}
%lineage_input /skatt/unrelated {unrelated.schema.json()}

In [5]:
# This will be done automatically by spark.write.path

%lineage_output /skatt/innskudd {innskudd.schema.json()}

In [6]:
# This is for debug purposes

%lineage_tree

Input datasets:
 |-- /skatt/person
 |-- /skatt/konto
 |-- /skatt/unrelated
Output datasets:
 |-- /skatt/innskudd


In [7]:
# Show GUI for mapping lineage fields

%lineage_fields innskudd

Accordion(children=(VBox(children=(HTML(value='<style>.widget-checkbox-label-bold > label > span {font-weight:…

In [8]:
# This will be the lineage output from GUI

innskudd.lineage

{'lineage': {'name': 'spark_schema',
  'type': 'structure',
  'fields': [{'name': 'kontonummer',
    'type': 'inherited',
    'confidence': 0.9,
    'sources': [{'field': 'kontonummer',
      'path': '/skatt/person',
      'version': 1600430243658},
     {'field': 'kontonummer',
      'path': '/skatt/konto',
      'version': 1600430243658}]},
   {'name': 'personidentifikator',
    'type': 'inherited',
    'confidence': 0.9,
    'sources': [{'field': 'personidentifikator',
      'path': '/skatt/person',
      'version': 1600430243658}]},
   {'name': 'innskudd',
    'type': 'inherited',
    'confidence': 0.9,
    'sources': [{'field': 'innskudd',
      'path': '/skatt/konto',
      'version': 1600430243658}]}],
  'sources': [{'path': '/skatt/person', 'version': 1600430243658},
   {'path': '/skatt/konto', 'version': 1600430243658},
   {'path': '/skatt/unrelated', 'version': 1600430243658}]}}

In [9]:
# Return simple lineage template by path

%lineage_json --path /skatt/innskudd

{'lineage': {'name': 'spark_schema',
  'type': 'structure',
  'fields': [{'name': 'kontonummer',
    'type': 'inherited',
    'confidence': 0.9,
    'sources': [{'field': 'kontonummer',
      'path': '/skatt/person',
      'version': 1600430243658},
     {'field': 'kontonummer',
      'path': '/skatt/konto',
      'version': 1600430243658}]},
   {'name': 'personidentifikator',
    'type': 'inherited',
    'confidence': 0.9,
    'sources': [{'field': 'personidentifikator',
      'path': '/skatt/person',
      'version': 1600430243658}]},
   {'name': 'innskudd',
    'type': 'inherited',
    'confidence': 0.9,
    'sources': [{'field': 'innskudd',
      'path': '/skatt/konto',
      'version': 1600430243658}]}],
  'sources': [{'path': '/skatt/person', 'version': 1600430243658},
   {'path': '/skatt/konto', 'version': 1600430243658},
   {'path': '/skatt/unrelated', 'version': 1600430243658}]}}